This notebook gives Private score of 0.517 (Rank ~27 to ~38 out of 1707) and Public score of 0.510  

https://www.kaggle.com/c/favorita-grocery-sales-forecasting/


In [1]:
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

from scipy.stats import skew
from scipy.stats import hmean
from scipy.stats import kurtosis
from sklearn import preprocessing

In [2]:
df_train = pd.read_csv(
    '../data/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # From: 2016-01-01 To: 2017-08-15
)

In [3]:
print(df_train.shape)
df_train.head()

(59038132, 5)


,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [4]:
df_test = pd.read_csv(
    "../data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [5]:
df_test.head()

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [6]:
items = pd.read_csv(
    "../data/items.csv",
).set_index("item_nbr")
items['item_nbrs'] = items.index

In [7]:
items.head()

,family,class,perishable,item_nbrs
item_nbr,,,,
96995,GROCERY I,1093,0,96995
99197,GROCERY I,1067,0,99197
103501,CLEANING,3008,0,103501
103520,GROCERY I,1028,0,103520
103665,BREAD/BAKERY,2712,1,103665


In [8]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
print(df_2017.shape)
del df_train

(23808261, 5)


In [9]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [12]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train


In [13]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)


In [14]:
items = items.reindex(df_2017.index.get_level_values(1))


In [15]:
le = preprocessing.LabelEncoder()
le.fit(items.family)

LabelEncoder()

In [16]:
items['family_cat'] = le.transform(items.family)

In [17]:
items.head()

,family,class,perishable,item_nbrs,family_cat
item_nbr,,,,,
96995,GROCERY I,1093,0,96995,12
99197,GROCERY I,1067,0,99197,12
103520,GROCERY I,1028,0,103520,12
103665,BREAD/BAKERY,2712,1,103665,5
105574,GROCERY I,1045,0,105574,12


In [18]:
df_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.000000     ...        1.098612   
          99197       0.000000    0.693147     ...        0.000000   
          103520      0.693147    0.693147     ...        0.000000   
          103665      0.000000    2.079442     ...        0.693147   
          105574      1.386294    2.302585     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [19]:
stores = pd.read_csv("../data/stores.csv").set_index("store_nbr")
stores['stores_nbrs'] = stores.index

In [20]:
stores.head()

,city,state,type,cluster,stores_nbrs
store_nbr,,,,,
1,Quito,Pichincha,D,13,1
2,Quito,Pichincha,D,13,2
3,Quito,Pichincha,D,8,3
4,Quito,Pichincha,D,9,4
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,5


In [21]:
stores = stores.reindex(df_2017.index.get_level_values(0))

In [22]:
stores.shape

(167515, 5)

In [23]:
stores.head()

,city,state,type,cluster,stores_nbrs
store_nbr,,,,,
1,Quito,Pichincha,D,13,1
1,Quito,Pichincha,D,13,1
1,Quito,Pichincha,D,13,1
1,Quito,Pichincha,D,13,1
1,Quito,Pichincha,D,13,1


In [24]:
le = preprocessing.LabelEncoder()
le.fit(stores.city)
stores['city_cat'] = le.transform(stores.city)

In [25]:
le = preprocessing.LabelEncoder()
le.fit(stores.type)
stores['type_cat'] = le.transform(stores.type)

In [26]:
le = preprocessing.LabelEncoder()
le.fit(stores.state)
stores['state_cat'] = le.transform(stores.state)

In [27]:
stores.head()

,city,state,type,cluster,stores_nbrs,city_cat,type_cat,state_cat
store_nbr,,,,,,,,
1,Quito,Pichincha,D,13,1,18,3,12
1,Quito,Pichincha,D,13,1,18,3,12
1,Quito,Pichincha,D,13,1,18,3,12
1,Quito,Pichincha,D,13,1,18,3,12
1,Quito,Pichincha,D,13,1,18,3,12


In [29]:
def get_nearwd(date,b_date):
    date_list = pd.date_range(date-timedelta(140),periods=21,freq='7D').date
    result = date_list[date_list<=b_date][-1]
    return result

def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        
        "median_3_2017": get_timespan(df_2017, t2017, 3, 3).median(axis=1).values,
        "median_7_2017": get_timespan(df_2017, t2017, 7, 7).median(axis=1).values,
        "median_14_2017": get_timespan(df_2017, t2017, 14, 14).median(axis=1).values,
        "median_30_2017": get_timespan(df_2017, t2017, 30, 30).median(axis=1).values,
        "median_60_2017": get_timespan(df_2017, t2017, 60, 60).median(axis=1).values,
        "median_140_2017": get_timespan(df_2017, t2017, 140, 140).median(axis=1).values,
        
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        
        "min_3_2017": get_timespan(df_2017, t2017, 3, 3).min(axis=1).values,
        "min_7_2017": get_timespan(df_2017, t2017, 7, 7).min(axis=1).values,
        "min_14_2017": get_timespan(df_2017, t2017, 14, 14).min(axis=1).values,
        "min_30_2017": get_timespan(df_2017, t2017, 30, 30).min(axis=1).values,
        "min_60_2017": get_timespan(df_2017, t2017, 60, 60).min(axis=1).values,
        "min_140_2017": get_timespan(df_2017, t2017, 140, 140).min(axis=1).values,
        
        "max_3_2017": get_timespan(df_2017, t2017, 3, 3).max(axis=1).values,
        "max_7_2017": get_timespan(df_2017, t2017, 7, 7).max(axis=1).values,
        "max_14_2017": get_timespan(df_2017, t2017, 14, 14).max(axis=1).values,
        "max_30_2017": get_timespan(df_2017, t2017, 30, 30).max(axis=1).values,
        "max_60_2017": get_timespan(df_2017, t2017, 60, 60).max(axis=1).values,
        "max_140_2017": get_timespan(df_2017, t2017, 140, 140).max(axis=1).values,
        
        "skew_3_2017": get_timespan(df_2017, t2017, 3, 3).skew(axis=1).values,
        "skew_7_2017": get_timespan(df_2017, t2017, 7, 7).skew(axis=1).values,
        "skew_14_2017": get_timespan(df_2017, t2017, 14, 14).skew(axis=1).values,
        "skew_30_2017": get_timespan(df_2017, t2017, 30, 30).skew(axis=1).values,
        "skew_60_2017": get_timespan(df_2017, t2017, 60, 60).skew(axis=1).values,
        "skew_140_2017": get_timespan(df_2017, t2017, 140, 140).skew(axis=1).values,
        
        "std_3_2017": get_timespan(df_2017, t2017, 3, 3).std(axis=1).values,
        "std_7_2017": get_timespan(df_2017, t2017, 7, 7).std(axis=1).values,
        "std_14_2017": get_timespan(df_2017, t2017, 14, 14).std(axis=1).values,        
        "std_30_2017": get_timespan(df_2017, t2017, 30, 30).std(axis=1).values,        
        "std_60_2017": get_timespan(df_2017, t2017, 60, 60).std(axis=1).values,        
        "std_140_2017": get_timespan(df_2017, t2017, 140, 140).std(axis=1).values,        
        
        "kurtosis_3_2017": get_timespan(df_2017, t2017, 3, 3).kurtosis(axis=1).values,
        "kurtosis_7_2017": get_timespan(df_2017, t2017, 7, 7).kurtosis(axis=1).values,
        "kurtosis_14_2017": get_timespan(df_2017, t2017, 14, 14).kurtosis(axis=1).values,
        "kurtosis_30_2017": get_timespan(df_2017, t2017, 30, 30).kurtosis(axis=1).values,
        "kurtosis_60_2017": get_timespan(df_2017, t2017, 60, 60).kurtosis(axis=1).values,
        "kurtosis_140_2017": get_timespan(df_2017, t2017, 140, 140).kurtosis(axis=1).values,
        
        
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values, 
        
        "unpromo_16aftsum_2017":(1-get_timespan(promo_2017, t2017+timedelta(16), 16, 16)).iloc[:,1:].sum(axis=1).values,
        
    })
    
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
        
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values        
        
        date = get_nearwd(t2017+timedelta(i),t2017)
        ahead = (t2017-date).days
        if ahead!=0:
            X['ahead0_{}'.format(i)] = get_timespan(df_2017, date+timedelta(ahead), ahead, ahead).mean(axis=1).values
            X['ahead7_{}'.format(i)] = get_timespan(df_2017, date+timedelta(ahead), ahead+7, ahead+7).mean(axis=1).values
        X["day_1_2017_{}1".format(i)]= get_timespan(df_2017, date, 1, 1).values.ravel()
        X["day_1_2017_{}2".format(i)]= get_timespan(df_2017, date-timedelta(7), 1, 1).values.ravel()
        for m in [3,7,14,30,60,140]:
            X["mean_{}_2017_{}1".format(m,i)]= get_timespan(df_2017, date,m, m).\
                mean(axis=1).values
            X["mean_{}_2017_{}2".format(m,i)]= get_timespan(df_2017, date-timedelta(7),m, m).\
                mean(axis=1).values
    
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

**With Validation**

In [30]:
t2017 = date(2017, 6, 14)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
#X_val2,y_val2 = prepare_dataset(date(2017, 7, 12))
X_val, y_val = prepare_dataset(date(2017, 7, 26))
#X_val, y_val = prepare_dataset(date(2017, 7, 31))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [31]:
X_train.shape

(1005090, 187)

In [32]:
X_train['family_cat'] = pd.concat([items["family_cat"]] * 6).values
X_train['class_cat'] = pd.concat([items['class']] * 6).values
X_train['cluster'] = pd.concat([stores['cluster']] * 6).values
X_train['city_cat'] = pd.concat([stores['city_cat']] * 6).values
X_train['type_cat'] = pd.concat([stores['type_cat']] * 6).values
#X_train['state_cat'] = pd.concat([stores['state_cat']] * 6).values
X_train['item_nbrs'] = pd.concat([items['item_nbrs']] * 6).values
X_train['store_nbrs'] = pd.concat([stores['stores_nbrs']] * 6).values

In [33]:
X_val['family_cat'] = items["family_cat"].values
X_val['class_cat'] = items['class'].values
X_val['cluster'] = stores['cluster'].values
X_val['city_cat'] = stores['city_cat'].values
X_val['type_cat'] = stores['type_cat'].values
#X_val['state_cat'] = stores['state_cat'].values
X_val['item_nbrs'] = items['item_nbrs'].values
X_val['store_nbrs'] = stores['stores_nbrs'].values

In [34]:
X_val.shape

(167515, 194)

In [35]:
X_val = pd.get_dummies( X_val, columns = ['family_cat','cluster','city_cat','type_cat','class_cat','store_nbrs'] )


In [36]:
X_val.shape

(167515, 653)

In [37]:
X_train = pd.get_dummies( X_train, columns = ['family_cat','cluster','city_cat','type_cat','class_cat', 'store_nbrs'] )
X_train.shape

(1005090, 653)

In [38]:
#print(len(X_train.columns),list(X_train.columns))

In [39]:
print("Training and predicting models...")

params = {
    'num_leaves': 33,
    'objective': 'regression',
    'min_data_in_leaf': 250,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

Training and predicting models...


Validation mse: 0.361188824525 

In [ ]:
MAX_ROUNDS = 500
MAX_ROUNDS = 1000
MAX_ROUNDS = 5000
#MAX_ROUNDS = 300
val_pred = []
test_pred = []
cate_vars = [] #['family_cat', 'cluster','type_cat','city_cat']
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
    )
    #print("\n".join(("%s: %.2f" % x) for x in sorted(
    #    zip(X_train.columns, bst.feature_importance("gain")),
    #    key=lambda x: x[1], reverse=True
    #)))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))


In [ ]:
Validation mse: 0.359156566775  -- 


In [49]:
n_public = 5
weights=pd.concat([items["perishable"]]) * 0.25 + 1
print("Unweighted validation rmse: ", mean_squared_error(
   y_val, np.array(val_pred).transpose())**0.5)
print("Full validation rmse:       ", mean_squared_error(
   y_val, np.array(val_pred).transpose(), sample_weight=weights)**0.5)
print("'Public' validation rmse:   ", mean_squared_error(
   np.array(y_val)[:,:n_public], np.array(val_pred).transpose()[:,:n_public], sample_weight=weights)**0.5)
print("'Private' validation rmse:  ", mean_squared_error(
   np.array(y_val)[:,n_public:], np.array(val_pred).transpose()[:,n_public:], sample_weight=weights)**0.5)

Unweighted validation rmse:  0.599296726818
Full validation rmse:        0.598867485781
'Public' validation rmse:    0.574710675717
'Private' validation rmse:   0.609531447805


In [44]:
"""
#Change validation and then submit
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 10000)
submission.to_csv('lgbm_last.csv', float_format='%.5f', index=None)
"""

Making submission...


**No Validation**

In [36]:
print("Preparing dataset...")
t2017 = date(2017, 6, 21)
X_l, y_l = [], []
for i in range(6):
    #print(i)
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
#X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)


Preparing dataset...


In [38]:
X_train.shape

(1005090, 187)

In [39]:
X_train['family_cat'] = pd.concat([items["family_cat"]] * 6).values
X_train['class_cat'] = pd.concat([items['class']] * 6).values
X_train['cluster'] = pd.concat([stores['cluster']] * 6).values
X_train['city_cat'] = pd.concat([stores['city_cat']] * 6).values
X_train['type_cat'] = pd.concat([stores['type_cat']] * 6).values
#X_train['state_cat'] = pd.concat([stores['state_cat']] * 6).values
X_train['item_nbrs'] = pd.concat([items['item_nbrs']] * 6).values
X_train['store_nbrs'] = pd.concat([stores['stores_nbrs']] * 6).values

In [40]:
X_train = pd.get_dummies( X_train, columns = ['family_cat','cluster','city_cat','type_cat','class_cat', 'store_nbrs'] )
X_train.shape

(1005090, 653)

In [41]:
X_test.shape, X_train.shape

((167515, 187), (1005090, 653))

In [42]:
X_test['family_cat'] = items["family_cat"].values
X_test['class_cat'] = items['class'].values
X_test['cluster'] = stores['cluster'].values
X_test['city_cat'] = stores['city_cat'].values
X_test['type_cat'] = stores['type_cat'].values
#X_train['state_cat'] = pd.concat([stores['state_cat']] ).values
X_test['item_nbrs'] = items['item_nbrs'].values
X_test['store_nbrs'] = stores['stores_nbrs'].values

In [43]:
X_test.shape, X_train.shape

((167515, 194), (1005090, 653))

In [44]:
X_test = pd.get_dummies( X_test, columns = ['family_cat','cluster','city_cat','type_cat','class_cat', 'store_nbrs'] )
X_test.shape, X_train.shape

((167515, 653), (1005090, 653))

In [45]:
print("Training and predicting models...")
params = {
    'num_leaves': 33,
    'objective': 'regression',
    'min_data_in_leaf': 250,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

Training and predicting models...


In [46]:
MAX_ROUNDS = 500
MAX_ROUNDS = 1000
MAX_ROUNDS = 3200
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    #dval = lgb.Dataset(
    #    X_val, label=y_val[:, i], reference=dtrain,
    #    weight=items["perishable"] * 0.25 + 1,
    #    categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        #valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
        #valid_sets=[dtrain], early_stopping_rounds=50, 
        verbose_eval=100
    )
    #print("\n".join(("%s: %.2f" % x) for x in sorted(
    #    zip(X_train.columns, bst.feature_importance("gain")),
    #    key=lambda x: x[1], reverse=True
    #)))
    #val_pred.append(bst.predict(
    #    X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

#print("Validation mse:", mean_squared_error(
#    y_val, np.array(val_pred).transpose()))


Step 1


/home/ubuntu/.local/lib/python3.5/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10
Step 11
Step 12
Step 13
Step 14
Step 15
Step 16


In [47]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb_bugfixed.csv', float_format='%.5f', index=None)

Making submission...


Ref: 
https://www.kaggle.com/ceshine/lgbm-starter
https://www.kaggle.com/tunguz/lgbm-one-step-ahead-c8de0f